In [1]:
import requests
import pandas as pd
import re
import numpy as np
import os

import math
import statistics

from bs4 import BeautifulSoup

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
matplotlib.rcParams['pdf.fonttype'] = 42

%matplotlib inline

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0'}

# Scrape information for one therapist

In [2]:
page= 1 
url = f'https://www.psychologytoday.com/us/therapists/ny/new-york?sid=5d1f98636faae&rec_next={page}'
response = requests.get(url, headers=headers)
doc = BeautifulSoup(response.text)

rows = doc.find_all(class_='result-row')
name = rows[0].find(itemprop='name').text.strip()
url = rows[0].find(itemprop='url')['href']
description = rows[0].find(itemprop='description').text.strip()
title = rows[0].find(class_='result-suffix').text.replace('\n', '')
phone = rows[0].find(class_='result-phone').text.strip()
zipcode = rows[0].find(itemprop='postalCode').text.strip()

# Scrape the first page of results for therapists in NYC

In [3]:
page= 1 
url = f'https://www.psychologytoday.com/us/therapists/ny/new-york?sid=5d1f98636faae&rec_next={page}'
response = requests.get(url, headers=headers)
doc = BeautifulSoup(response.text)

therapists = doc.find_all(class_='result-row')
rows = []

for therapist in therapists:
    row = {}
    row['name'] = therapist.find(itemprop='name').text.strip()
    row['url'] = therapist.find(itemprop='url')['href']
    row['description'] = therapist.find(itemprop='description').text.strip()
    row['title'] = therapist.find(class_='result-suffix').text.replace('\n', '')
    row['phone'] = therapist.find(class_='result-phone').text.strip()
    row['zipcode'] = therapist.find(itemprop='postalCode').text.strip()
    rows.append(row)
rows

[{'name': 'Manhattan Mental Health Counseling',
  'url': 'https://www.psychologytoday.com/us/therapists/ny/new-york/241735?sid=5d1f98636faae&ref=1&rec_next=1&tr=ResultsName',
  'description': '"Welcome to Manhattan Mental Health Counseling. Here we value your decision to courageously seek help towards healing and thriving. We believe that if you take the step towards treatment, then as healers, we have the responsibility to make sure you are well cared for."',
  'title': 'Counselor,LMHC',
  'phone': '(212) 960-8626',
  'zipcode': '10018'},
 {'name': 'Ellen S. Daniels',
  'url': 'https://www.psychologytoday.com/us/therapists/ny/new-york/80412?sid=5d1f98636faae&ref=2&rec_next=1&tr=ResultsName',
  'description': '"Why do people seek therapy? Usually it\'s because they\'re feeling dissatisfied, anxious, or depressed--and it\'s becoming difficult to cope.  Do you feel stuck in a relationship or work situation?  Struggling with life transitions: marriage, divorce, parenting, career change, b

# Scrape all pages

In [4]:
results_per_page = 20
last_page = 6101
page_count = round(last_page / results_per_page)
pages = list(range(page_count))

endpoint = 1
endpoints = [1]
for page in pages:
    endpoint += 20
    endpoints.append(endpoint)
    
## Rename the list of endpoints
pages = endpoints

In [5]:
rows = []
errors=[]

for page in pages:
    url = f'https://www.psychologytoday.com/us/therapists/ny/new-york?sid=5d1f98636faae&rec_next={page}'
    response = requests.get(url, headers=headers)
    doc = BeautifulSoup(response.text)

    therapists = doc.find_all(class_='result-row')
    for therapist in therapists:
        try:
            row = {}
            row['name'] = therapist.find(itemprop='name').text.strip()
            row['url'] = therapist.find(itemprop='url')['href']
            row['description'] = therapist.find(itemprop='description').text.strip()
            row['phone'] = therapist.find(class_='result-phone').text.strip()
            row['zipcode'] = therapist.find(itemprop='postalCode').text.strip()

            try:
                row['title'] = therapist.find(class_='result-suffix').text.replace('\n', '')
            except:
                row['title'] = 'No title'
        except:
                errors.append(therapist)
                print(therapist.find(itemprop='name').text.strip())
        rows.append(row)
rows

KeyboardInterrupt: 

# Transform the list of dictionaries into dataframe & csv

In [ ]:
df_basic_info = pd.DataFrame(rows)

In [ ]:
df_basic_info.to_csv('nyc_therapists.csv', index=False)

# Scrape more data for each therapist

### Try with one therapist

In [ ]:
# url = 'https://www.psychologytoday.com/us/therapists/ny/new-york/241735?sid=5d1f98636faae&ref=1&rec_next=1&tr=ResultsName'
# response = requests.get(url, headers=headers)
# doc = BeautifulSoup(response.text)
# name = doc.find(itemprop='name').text.strip()

# name = doc.find(itemprop='name').text.strip()
# yrs_in_practice = doc.find(text='Years in Practice:').next_element.strip()
# school = doc.find(text='School:').next_element.strip()
# grad_yr = doc.find(text='Year Graduated:').next_element.strip()
# license_state = doc.find(text='License and State:').next_element.strip()
# specialities = doc.find(class_='spec-list attributes-top').text.strip()
# issues = doc.find(class_='attributes-issues').find(class_='col-split-md-2').text.strip()
# mental_health = doc.find(class_='attributes-mental-health').find(class_='col-split-md-2').text.strip()
# sexuality = doc.find(class_='attributes-sexuality').find(class_='col-split-md-2').text.strip()
# race_focus = doc.find(class_='attributes-ethnicity-focus').text.strip().split('Ethnicity:\n')[1]
# lang = doc.find(class_='attributes-language').text.strip().split('I also speak:\n')[1]
# age_focus = doc.find(class_='attributes-age-focus').text.strip().split('Age')[1].strip()
# community_focus = doc.find(class_='spec-list attributes-categories').text.split('Communities\n')[1].strip()
# therapy_types = doc.find(class_='attributes-treatment-orientation').find('div').text.strip()

In [ ]:
therapist_info = []
for row in rows:
    url = row['url']
    response = requests.get(url, headers=headers)
    doc = BeautifulSoup(response.text)
    name = doc.find(itemprop='name').text.strip()
    try:
        row['yrs_in_practice'] = doc.find(text='Years in Practice:').next_element.strip()
    except:
        row['yrs_in_practice'] = 'None'
    try:
        row['school'] = doc.find(text='School:').next_element.strip()
    except:
        row['school'] = 'None'
    try:
        row['grad_yr'] = doc.find(text='Year Graduated:').next_element.strip()
    except:
        row['grad_yr'] = np.nan
    try:
        row['license_state'] = doc.find(text='License and State:').next_element.strip()
    except:
        row['license_state'] = 'None'
    try:
        row['specialities'] = doc.find(class_='spec-list attributes-top').text.strip()
    except:
        row['specialities'] = 'None'
    try:
        row['issues'] = doc.find(class_='attributes-issues').find(class_='col-split-md-2').text.strip()
    except:
        row['issues'] = 'None'
    try:
        row['mental_health'] = doc.find(class_='attributes-mental-health').find(class_='col-split-md-2').text.strip()
    except:
        row['mental_health'] = 'None'
    try:
        row['sexuality'] = doc.find(class_='attributes-sexuality').find(class_='col-split-md-2').text.strip()
    except:
        row['sexuality'] = 'None'
    try:
        row['race_focus'] = doc.find(class_='attributes-ethnicity-focus').text.strip().split('Ethnicity:\n')[1]
    except:
        row['race_focus'] = 'None'
    try:
        row['lang'] = doc.find(class_='attributes-language').text.strip().split('I also speak:\n')[1]
    except:
        row['lang'] = 'None'
    try:
        row['age_focus'] = doc.find(class_='attributes-age-focus').text.strip().split('Age')[1].strip()
    except:
        row['age_focus'] = 'None'
    try:
        row['community_focus'] = doc.find(class_='spec-list attributes-categories').text.split('Communities\n')[1].strip()
    except:
        row['community_focus'] = 'None'
    try:
        row['therapy_types'] = doc.find(class_='attributes-treatment-orientation').find('div').text.strip()
    except:
        row['therapy_types'] = 'None'
    
    therapist_info.append(row)
    
therapist_info

In [ ]:
len(therapist_info)

# Save scraped ata as csv

In [ ]:
df = pd.DataFrame(therapist_info)
df.to_csv('nyc_therapists_complete.csv', index=False)